In [2]:
import pandas as pd
import random
from tqdm import tqdm
import numpy as np

In [2]:
train = pd.read_parquet("./clean_dataset/clean_60k.parquet")
print(train.shape)

(60347, 9)


In [3]:
special_values = [None, np.nan, ""]

for index, row in tqdm(train.iterrows(), total=len(train)):
    if row.isna().any() == True:
        column_names = ["A", "B", "C", "D", "E"]
        for column_name in column_names:
            value = train.loc[index, column_name]
            if (value in special_values) | pd.isna(value) | pd.isnull(value) | (value is None):
                answer = train.loc[index, "answer"]
                column_names.remove(answer)
                column_names.remove(column_name)
                # print(column_names)
                random_value1 = random.choice(column_names)
                column_names.remove(random_value1)
                
                # print(random_value)
                train.loc[index, column_name] = train.loc[index, random_value1]

100%|██████████| 60347/60347 [00:28<00:00, 2144.48it/s]


In [4]:
print(train.isnull().any(axis=1).sum())

0


In [5]:
train.to_parquet("./clean_dataset/fillna_clean_60k.parquet")

In [16]:
sciencemcq = pd.read_parquet("./sciencemcq.parquet")
sciencemcq.head()

,question,distractor3,distractor1,distractor2,correct_answer,support,embedding
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ...","[-0.11928173, -0.47883707, -0.37879574, -0.221..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...,"[0.039890513, -0.5017906, -0.26410213, 0.10449..."
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...,"[0.050477862, -0.1211036, -0.293591, -0.148012..."
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...,"[-0.20777017, -0.32109252, -0.38465917, 0.0218..."
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...,"[0.23974684, -0.64920425, -0.3446707, 0.396083..."


In [17]:
sciencemcq.drop("embedding", axis=1, inplace=True)
sciencemcq.head()

,question,distractor3,distractor1,distractor2,correct_answer,support
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


In [18]:
sciencemcq = sciencemcq.rename(
    columns={
        "question": "prompt",
        "distractor1": "A",
        "distractor2": "B",
        "distractor3": "C",
        "correct_answer": "D",
        "support": "context"
    }
)
sciencemcq.head()

,prompt,C,A,B,D,context
0,What type of organism is commonly used in prep...,viruses,protozoa,gymnosperms,mesophilic organisms,"Mesophiles grow best in moderate temperature, ..."
1,What phenomenon makes global winds blow northe...,tropical effect,muon effect,centrifugal effect,coriolis effect,Without Coriolis Effect the global winds would...
2,Changes from a less-ordered state to a more-or...,endothermic,unbalanced,reactive,exothermic,Summary Changes of state are examples of phase...
3,What is the least dangerous radioactive decay?,zeta decay,beta decay,gamma decay,alpha decay,All radioactive decay is dangerous to living t...
4,Kilauea in hawaii is the world’s most continuo...,magma,greenhouse gases,carbon and smog,smoke and ash,Example 3.5 Calculating Projectile Motion: Hot...


In [19]:
new_column_order = ["prompt", "context", "A", "B", "C", "D"]
sciencemcq = sciencemcq[new_column_order]
sciencemcq.head()

,prompt,context,A,B,C,D
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",protozoa,gymnosperms,viruses,mesophilic organisms
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,muon effect,centrifugal effect,tropical effect,coriolis effect
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,unbalanced,reactive,endothermic,exothermic
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,beta decay,gamma decay,zeta decay,alpha decay
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,greenhouse gases,carbon and smog,magma,smoke and ash


In [20]:
sciencemcq["answer"] = "D"
sciencemcq.head()

,prompt,context,A,B,C,D,answer
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",protozoa,gymnosperms,viruses,mesophilic organisms,D
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,muon effect,centrifugal effect,tropical effect,coriolis effect,D
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,unbalanced,reactive,endothermic,exothermic,D
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,beta decay,gamma decay,zeta decay,alpha decay,D
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,greenhouse gases,carbon and smog,magma,smoke and ash,D


In [21]:
sciencemcq.shape

(11679, 7)

In [22]:
sciencemcq.insert(6, "E", None)
sciencemcq.head()

,prompt,context,A,B,C,D,E,answer
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",protozoa,gymnosperms,viruses,mesophilic organisms,None,D
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,muon effect,centrifugal effect,tropical effect,coriolis effect,None,D
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,unbalanced,reactive,endothermic,exothermic,None,D
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,beta decay,gamma decay,zeta decay,alpha decay,None,D
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,greenhouse gases,carbon and smog,magma,smoke and ash,None,D


In [23]:
# special_values = [None, np.nan, ""]

# for index, row in tqdm(sciencemcq.iterrows(), total=len(sciencemcq)):
#     if row.isna().any() == True:
#         column_names = ["A", "B", "C", "D", "E"]
#         for column_name in column_names:
#             value = sciencemcq.loc[index, column_name]
#             if (value in special_values) | pd.isna(value) | pd.isnull(value) | (value is None):
#                 answer = sciencemcq.loc[index, "answer"]
#                 column_names.remove(answer)
#                 column_names.remove(column_name)
#                 # print(column_names)
#                 random_value1 = random.choice(column_names)
#                 column_names.remove(random_value1)
                
#                 # print(random_value)
#                 sciencemcq.loc[index, column_name] = sciencemcq.loc[index, random_value1]

In [24]:
sciencemcq.head()

,prompt,context,A,B,C,D,E,answer
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",protozoa,gymnosperms,viruses,mesophilic organisms,None,D
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,muon effect,centrifugal effect,tropical effect,coriolis effect,None,D
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,unbalanced,reactive,endothermic,exothermic,None,D
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,beta decay,gamma decay,zeta decay,alpha decay,None,D
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,greenhouse gases,carbon and smog,magma,smoke and ash,None,D


In [25]:
for index, row in tqdm(sciencemcq.iterrows(), total=len(sciencemcq)):
    right_column = row["answer"]
    changeable_columns = ["A", "B", "C", "D", "E"]
    change_columns = random.choice(changeable_columns)
    D_value = sciencemcq.loc[index, right_column]
    change_value = sciencemcq.loc[index, change_columns]
    sciencemcq.loc[index, right_column] = change_value
    sciencemcq.loc[index, change_columns] = D_value
    sciencemcq.loc[index, "answer"] = change_columns

sciencemcq.head(10)

  0%|          | 0/11679 [00:00<?, ?it/s]

100%|██████████| 11679/11679 [00:06<00:00, 1673.79it/s]


,prompt,context,A,B,C,D,E,answer
0,What type of organism is commonly used in prep...,"Mesophiles grow best in moderate temperature, ...",protozoa,gymnosperms,mesophilic organisms,viruses,None,C
1,What phenomenon makes global winds blow northe...,Without Coriolis Effect the global winds would...,coriolis effect,centrifugal effect,tropical effect,muon effect,None,A
2,Changes from a less-ordered state to a more-or...,Summary Changes of state are examples of phase...,exothermic,reactive,endothermic,unbalanced,None,A
3,What is the least dangerous radioactive decay?,All radioactive decay is dangerous to living t...,beta decay,gamma decay,zeta decay,alpha decay,None,D
4,Kilauea in hawaii is the world’s most continuo...,Example 3.5 Calculating Projectile Motion: Hot...,smoke and ash,carbon and smog,magma,greenhouse gases,None,A
5,"When a meteoroid reaches earth, what is the re...","Meteoroids are smaller than asteroids, ranging...",comet,meteor,orbit,None,meteorite,E
6,What kind of a reaction occurs when a substanc...,A combustion reaction occurs when a substance ...,invention reaction,Fluid Reaction,combustion reaction,nitrogen reaction,None,C
7,Organisms categorized by what species descript...,Ring species Ring species demonstrate a versio...,surface species,fitting species,species complex,ring species,None,D
8,Alpha emission is a type of what?,One type of radioactivity is alpha emission. W...,radiation,heat,radioactivity,light,None,C
9,What is the stored food in a seed called?,The stored food in a seed is called endosperm ...,endosperm,membrane,larval,pollin,None,A


In [26]:
sciencemcq["answer"].value_counts()

answer
B    2375
A    2343
C    2330
E    2328
D    2303
Name: count, dtype: int64

In [27]:
sciencemcq.loc[0]["context"]

'Mesophiles grow best in moderate temperature, typically between 25°C and 40°C (77°F and 104°F). Mesophiles are often found living in or on the bodies of humans or other animals. The optimal growth temperature of many pathogenic mesophiles is 37°C (98°F), the normal human body temperature. Mesophilic organisms have important uses in food preparation, including cheese, yogurt, beer and wine.'

In [28]:
sciencemcq.to_csv("original_sciencemcq.csv", index=False)